# Merge z warunkami

Używając danych surowych z pliku **ceny_produktow_cleaned.csv**, wykonaj zadanie polegające na sprawdzeniu, ile razy dla danego produktu, województwa, miesiąca historycznie była notowana niższa cena towaru. W tym celu wykonaj następujące kroki:

1. połącz tabelę samą ze sobą. Jakiego typu joina należy użyć?
1. odfiltruj dane tak, aby znaleźć lata wcześniejsze oraz wartości mniejsze niż tej w danym roku, dla danego województwa,
1. odpowiednio pogrupuj dane.

Który produkt/y miał takich wystąpień najwięcej?

> Zobacz co się stanie z nazwami kolumn w momencie, gdy kolumny nie są użyte jako warunek łączenia, a mają te same nazwy.

In [1]:
import pandas as pd
p = "../../01_Dane/ceny_produktow_cleaned.csv"

df = pd.read_csv(
    p,
    sep=';',
    decimal='.'
)
df.head(1)

,province,product_types,currency,product_group_id,product_line,value,date,product
0,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg


In [2]:
df.shape[0]

95830

In [6]:
# laczenie tabeli samej ze soba, tak by dostac wpisy z przyszlosci i przeszlosci wzgledem badanego wpisu
# badany wpis - to ten dla ktorego w danej chwili merge szuka poleczenia
tmp_df = df.merge(df, how='left', on=['province', 'product'])
tmp_df.head(15)

In [7]:
tmp_df.shape[0]

22239696

In [11]:
tmp_df['date_x_dt'] = pd.to_datetime(tmp_df['date_x'])
tmp_df['date_y_dt'] = pd.to_datetime(tmp_df['date_y'])

In [12]:
tmp_df.head(3)

,province,product_types_x,currency_x,product_group_id_x,product_line_x,value_x,date_x,product,product_types_y,currency_y,product_group_id_y,product_line_y,value_y,date_y,date_x_dt,date_y_dt
0,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,2013-03-01,2013-03-01
1,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,15.76,2007-1,2013-03-01,2007-01-01
2,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,20.88,2019-10,2013-03-01,2019-10-01


In [13]:
w1 = tmp_df['date_x_dt'] > tmp_df['date_y_dt']
w2 = tmp_df['value_x'] > tmp_df['value_y']
tmp_df.loc[w1 & w2].shape[0]

7631121

In [14]:
# nadpisanie ramki danych
tmp_df = tmp_df.loc[w1 & w2]

In [17]:
tmp_df.head(5)

,province,product_types_x,currency_x,product_group_id_x,product_line_x,value_x,date_x,product,product_types_y,currency_y,product_group_id_y,product_line_y,value_y,date_y,date_x_dt,date_y_dt
1,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,15.76,2007-1,2013-03-01,2007-01-01
3,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,16.26,2010-4,2013-03-01,2010-04-01
4,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,17.52,2011-5,2013-03-01,2011-05-01
5,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,15.57,2003-8,2013-03-01,2003-08-01
7,PODKARPACKIE,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,21.37,2013-3,szynka wieprzowa gotowana - za 1kg,NaN,zł,2,szynka wieprzowa gotowana - za 1kg,19.47,2012-1,2013-03-01,2012-01-01


In [31]:
tmp_df.groupby(by=['product', 'province', 'value_x', 'date_x_dt']).size().sort_values()

product                                   province             value_x  date_x_dt 
kiszka kaszana jęczmienna - za 1kg        WARMIŃSKO-MAZURSKIE  3.17     1999-07-01      1
mięso wołowe z kością (rostbef) - za 1kg  MAŁOPOLSKIE          8.56     1999-06-01      1
kasza gryczana prażona cała - za 1kg      LUBUSKIE             3.26     2004-04-01      1
                                                               3.27     1999-05-01      1
                                                               3.31     1999-03-01      1
                                                                                     ... 
kiszka kaszana jęczmienna - za 1kg        WARMIŃSKO-MAZURSKIE  9.62     2019-11-01    249
sól warzona biała workowana - za 1kg      POLSKA               0.53     2019-11-01    249
boczek wędzony z żeberkami - za 1kg       WIELKOPOLSKIE        17.60    2019-12-01    250
                                          POLSKA               18.95    2019-12-01    250
salceson Włoski -

In [37]:
tmp_df.groupby(by=['product', 'province', 'value_x', 'date_x_dt']).count().sort_values(by='date_x_dt', ascending=False)[['value_y', 'date_y_dt']]

,,,,value_y,date_y_dt
product,province,value_x,date_x_dt,,
mięso wołowe bez kości (z udźca) - za 1kg,ZACHODNIOPOMORSKIE,32.81,2019-12-01,232,232
kurczęta patroszone - za 1kg,MAZOWIECKIE,5.24,2019-12-01,98,98
"mąka pszenna Poznańska, workowana - za 1kg",POLSKA,1.28,2019-12-01,223,223
salceson Włoski - za 1kg,WARMIŃSKO-MAZURSKIE,11.90,2019-12-01,244,244
kurczęta patroszone - za 1kg,ŚLĄSKIE,3.31,2019-12-01,5,5
...,...,...,...,...,...
szynka wieprzowa gotowana - za 1kg,WIELKOPOLSKIE,16.17,1999-02-01,1,1
filety z morszczuka mrożone - za 1kg,ZACHODNIOPOMORSKIE,9.41,1999-02-01,1,1
mięso wieprzowe z kością (schab środkowy) - za 1kg,POMORSKIE,12.28,1999-02-01,1,1


In [32]:
tmp_df.groupby(by=['product', 'province', 'date_x_dt']).size().sort_values()

product                                         province             date_x_dt 
baleron gotowany - za 1kg                       DOLNOŚLĄSKIE         1999-02-01      1
mieszanka mrożona marchew i groszek - za 1kg    WARMIŃSKO-MAZURSKIE  1999-10-01      1
                                                                     1999-07-01      1
chleb mieszany zwykły (pszenno-żytni) - za 1kg  LUBELSKIE            2003-05-01      1
                                                                     2003-08-01      1
                                                                                  ... 
kiszka kaszana jęczmienna - za 1kg              WARMIŃSKO-MAZURSKIE  2019-11-01    249
sól warzona biała workowana - za 1kg            POLSKA               2019-11-01    249
boczek wędzony z żeberkami - za 1kg             POLSKA               2019-12-01    250
                                                WIELKOPOLSKIE        2019-12-01    250
salceson Włoski - za 1kg                        MA

In [38]:
250/12

20.833333333333332